# Create ROR Derived Tables

Explodes ROR arrays from `openalex.institutions.ror` into normalized tables:
- `ror_relationships`: Parent/child/related relationships between ROR IDs
- `ror_types`: Institution types (education, company, funder, etc.)
- `ror_labels`: International name labels by language
- `ror_aliases`: Alias/alternate names

In [ ]:
%sql
-- ror_relationships: explode relationships array
-- Extracts parent/child/related relationships between institutions
CREATE OR REPLACE TABLE openalex.institutions.ror_relationships AS
SELECT
  REPLACE(id, 'https://ror.org/', '') AS ror_id,
  REPLACE(rel.id, 'https://ror.org/', '') AS related_ror_id,
  rel.type AS relationship_type
FROM openalex.institutions.ror
LATERAL VIEW EXPLODE(relationships) AS rel
WHERE rel.id IS NOT NULL

In [ ]:
%sql
-- Verify ror_relationships
SELECT relationship_type, COUNT(*) as cnt
FROM openalex.institutions.ror_relationships
GROUP BY relationship_type

In [ ]:
%sql
-- ror_types: explode types array
-- Each ROR record can have multiple types (e.g., education, funder)
CREATE OR REPLACE TABLE openalex.institutions.ror_types AS
SELECT
  REPLACE(id, 'https://ror.org/', '') AS ror_id,
  type_val AS type
FROM openalex.institutions.ror
LATERAL VIEW EXPLODE(types) AS type_val

In [ ]:
%sql
-- Verify ror_types
SELECT type, COUNT(*) as cnt
FROM openalex.institutions.ror_types
GROUP BY type
ORDER BY cnt DESC

In [ ]:
%sql
-- ror_labels: names with 'label' type (international display names)
CREATE OR REPLACE TABLE openalex.institutions.ror_labels AS
SELECT
  REPLACE(id, 'https://ror.org/', '') AS ror_id,
  n.lang AS iso639,
  n.value AS label
FROM openalex.institutions.ror
LATERAL VIEW EXPLODE(names) AS n
WHERE ARRAY_CONTAINS(n.types, 'label')

In [ ]:
%sql
-- Verify ror_labels
SELECT COUNT(*) as total_labels, COUNT(DISTINCT ror_id) as unique_rors
FROM openalex.institutions.ror_labels

In [ ]:
%sql
-- ror_aliases: names with 'alias' type (alternate names)
CREATE OR REPLACE TABLE openalex.institutions.ror_aliases AS
SELECT
  REPLACE(id, 'https://ror.org/', '') AS ror_id,
  n.value AS alias
FROM openalex.institutions.ror
LATERAL VIEW EXPLODE(names) AS n
WHERE ARRAY_CONTAINS(n.types, 'alias')

In [ ]:
%sql
-- Verify ror_aliases
SELECT COUNT(*) as total_aliases, COUNT(DISTINCT ror_id) as unique_rors
FROM openalex.institutions.ror_aliases

In [ ]:
%sql
-- Verification: Check UBC relationships (should have NO Parent relationships after ROR fix)
SELECT *
FROM openalex.institutions.ror_relationships
WHERE ror_id = '03rmrcq20'  -- UBC